## Data Transformation of Title Column

##### Translating the data from Polish language to English Language

In [3]:
# Importing required modules
import os
from dotenv import load_dotenv
import pandas as pd
from snowflake.sqlalchemy import URL
from sqlalchemy import create_engine
from snowflake.connector.pandas_tools import pd_writer
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe

In [4]:
# Loading the method to access environment variables from .env file
# Create the .env file and then initialize variables that holds the following:
# Snowflake account name, Snowfalke username, Snowflake password, google account email
# Also make sure you copy the service.json file (json file that we get by enabling api in google developer console) in this directory.
load_dotenv()

True

In [19]:
# Configuration for connecting to snowflake database
engine = create_engine(URL(
                    account = os.getenv('snowflake_account_name'),
                    user = os.getenv('snowflake_user_name'),
                    password = os.getenv('snowflake_password'),
                    database = 'realestate',
                    schema = 'public',
                    warehouse = 'realestate_wh'))

In [27]:
# A google sheet contains the transformed records will be created.
# Along with that log table is also created that wil keep record of the file name for the google sheet.
with engine.connect() as conn:
    try:
        query = """ SELECT RN, TITLE FROM otodom_data_flatten ORDER BY rn"""

        df = pd.read_sql(query,conn)

        gc = gspread.service_account('service_account.json') # json file that we get by enabling api in google developer console

        loop_counter = 0
        chunk_size = 320000
        file_name = 'OTODOM_ANALYSIS_'
        user_email = 'example@gmail.com' #google_account_email
        for i in range(0,len(df),chunk_size):
            loop_counter += 1
            df_in = df.iloc[i:(i+chunk_size), :]

            spreadsheet_title = file_name + str(loop_counter)
            try:
                locals()['sh'+str(loop_counter)] = gc.open(spreadsheet_title)
            except gspread.SpreadsheetNotFound:
                locals()['sh'+str(loop_counter)] = gc.create(spreadsheet_title)

            locals()['sh'+str(loop_counter)].share(user_email, perm_type='user', role='writer')
            wks = locals()['sh'+str(loop_counter)].get_worksheet(0)
            wks.resize(len(df_in)+1)
            set_with_dataframe(wks, df_in)   
                
            column = 'C'   # Column to apply the formula 
            start_row = 2  # Starting row to apply the formula
            end_row = wks.row_count   # Ending row to apply the formula
            cell_range = f'{column}{start_row}:{column}{end_row}' 
            curr_row = start_row
            cell_list = wks.range(cell_range)
            
            for cell in cell_list:
                cell.value = f'=GOOGLETRANSLATE(B{curr_row},"pl","en")'
                curr_row += 1
                
            # Update the worksheet with the modified cells
            wks.update_cells(cell_list, value_input_option='USER_ENTERED')

            df_log = pd.DataFrame({'ID':[loop_counter], 'SPREADSHEET_NAME':[spreadsheet_title]})
            df_log.to_sql('otodom_data_log', con=engine, if_exists='append', index=False, chunksize=320000, method=pd_writer)


    except Exception as e:
        print('Error',e)
    finally:
        conn.close()
engine.dispose()
# You can manually download and upload the data in snowflake database or use the below cell for uploading the directly from google sheets.

In [28]:
# Retrieving the file name from log table in snowflake and using it to retrieve the file from google sheets and upload the data to snowflake.
with engine.connect() as conn:
    try:
        query = """ SELECT ID, SPREADSHEET_NAME FROM otodom_data_log """
        df = pd.read_sql(query,conn)
        df.columns = map(lambda x: str(x).upper(), df.columns)

        gc = gspread.service_account('service_account.json')
        loop_counter = 0

        for index, row in df.iterrows():
            loop_counter += 1
            locals()['sh'+str(loop_counter)] = gc.open(row['SPREADSHEET_NAME'])
            wks = locals()['sh'+str(loop_counter)].get_worksheet(0)
            df_out = get_as_dataframe(wks, usecols=[0,1,2], nrows=wks.row_count, header=None, skiprows=1, evaluate_formulas=True)
            
            df_out.columns = ['RN', 'TITLE', 'TITLE_ENG']
            df_out.to_sql('otodom_data_transformed_title', con=engine, if_exists='append', index=False, chunksize=16000, method=pd_writer)

    except Exception as e:
        print('Error',e)
    finally:
        conn.close()
engine.dispose()